# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

c:\Users\franc\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
parquet_files



['../../05_src/data/prices\\A\\A_2000\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2003\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2004\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2005\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2006\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2007\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2008\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2009\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2010\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2011\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2012\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2013\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2014\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2015\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2016\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2017\\part.0.pa

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.
import dask.dataframe as dd
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))  
)
dd_shift2 = dd_shift.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Adj_Close_lag_1 = x['Adj Close'].shift(1)) 
)
dd_rets = dd_shift2.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1  #to calculate return
)
dd_feat = dd_rets.assign(
    hi_lo_range = lambda x: x['High'] - x['Low'] #to calculate the day's High minus Low
)
dd_feat

C:\Users\franc\AppData\Local\Temp\ipykernel_94664\1792864932.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
C:\Users\franc\AppData\Local\Temp\ipykernel_94664\1792864932.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift2 = dd_shift.groupby('Ticker', group_keys=False).apply(


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
npartitions=13078,,,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.
import pandas as pd
pandas_df=dd_feat.compute()
pandas_df["10_days_moving_avg"]=pandas_df["Returns"].rolling(10).mean()



Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
Ticker,,,,,,,,,,,,
DOV,2002-01-02 00:00:00+00:00,16.045656,24.842640,24.963299,24.420326,24.869452,1022622.0,2002,NaN,NaN,NaN,0.542973
DOV,2002-01-03 00:00:00+00:00,16.340076,25.298468,25.405722,24.835936,24.876156,989355.0,2002,24.842640,16.045656,0.018349,0.569786
DOV,2002-01-04 00:00:00+00:00,16.721081,25.888363,25.955397,25.405722,25.405722,1319488.0,2002,25.298468,16.340076,0.023317,0.549675
DOV,2002-01-07 00:00:00+00:00,16.608503,25.714075,26.136387,25.687262,25.975506,1060812.0,2002,25.888363,16.721081,-0.006732,0.449125
DOV,2002-01-08 00:00:00+00:00,16.236160,25.137587,25.807922,24.869452,25.673855,976377.0,2002,25.714075,16.608503,-0.022419,0.938471
...,...,...,...,...,...,...,...,...,...,...,...,...
CTLT,2011-12-23 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2011,NaN,NaN,NaN,NaN
CTLT,2011-12-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2011,NaN,NaN,NaN,NaN
CTLT,2011-12-28 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2011,NaN,NaN,NaN,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)
- No
- It depends on the dataset size. If we have a smaller dataset, pandas might be faster. But Desk is better if it's a large dataset.  

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.